In [15]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import lightgbm as lgb
from catboost import CatBoostClassifier
%matplotlib inline

In [131]:
features = ["Age", "Pclass", "Sex_male", "Family", "Alone", "Fare", "Embarked_S", "Embarked_C", "Embarked_Q"]

def data_preprocessing(train, test):

    dataset = pd.concat([train, test], axis=0)

    # 家族人数を表すカラムを追加
    dataset['Family'] = dataset['SibSp'] + dataset['Parch']
    dataset.loc[dataset['Family'] == 0, 'Alone'] = 1
    dataset['Alone'] = dataset['Alone'].fillna(value=0)

    # Age, honorificの処理 *********************************************************************************
    # 最大分割回数 = 2で、名前を","または"."で分割
    dataset['honorific'] = dataset['Name'].str.split('[,.]', 2, expand = True)[1].str.strip()

    # データ可視化用の処理
    #hist_honorific = dataset[['honorific', 'Survived', 'PassengerId']].groupby(['honorific', 'Survived']).count().unstack()
    #hist_honorific.plot.bar(stacked=True)

    # 敬称でgroupbyし、平均年齢を算出
    average_age = dataset[['honorific', 'Age']].groupby(['honorific']).mean()

    # 置き換え用の一時カラム'abc'に、敬称に基づいた平均年齢を格納
    dataset['abc'] = dataset['honorific'].apply(lambda x: average_age.loc[x,'Age'])
    # 年齢がNaNについては敬称に基づいた平均年齢で補完
    dataset['Age'] = dataset['Age'].fillna(dataset['abc'])
    # 置き換え用の一時カラムを削除
    dataset = dataset.drop(columns = ['abc'])

    # 重要そうな敬称以外は'Other'に統一
    dataset['honorific'] = dataset['honorific'].where((dataset['honorific'] == 'Mr') | (dataset['honorific'] == 'Miss') | (dataset['honorific'] == 'Mrs') | (dataset['honorific'] == 'Master'), other = 'Other')
    
    from sklearn.preprocessing import LabelEncoder
    encoder = LabelEncoder()
    dataset['honorific'] = encoder.fit_transform(dataset['honorific'])
    # **********************************************************************************************

    # Ageの欠損値にはpandaではなくscikit-learnのライブラリを使用
    from sklearn.impute import SimpleImputer
    imputer = SimpleImputer(missing_values = np.nan, strategy = 'median')
    dataset['Fare'] = imputer.fit_transform(dataset['Fare'].values.reshape(-1,1))
    imputer = SimpleImputer(missing_values = np.nan, strategy = 'most_frequent')
    dataset['Embarked'] = imputer.fit_transform(dataset['Embarked'].values.reshape(-1,1))

    dataset = pd.get_dummies(dataset, columns=['Embarked'], drop_first=False)
    # SexはLabelEncodingを用いると多重共線性が発生? OneHot Encodingへ変更
    dataset = pd.get_dummies(dataset, columns=['Sex'], drop_first=True)

    # Features Scaling
    from sklearn.preprocessing import StandardScaler
    scaler = StandardScaler()
    dataset['Age'] = scaler.fit_transform(dataset['Age'].values.reshape(-1,1))
    dataset['Fare'] = scaler.fit_transform(dataset['Fare'].values.reshape(-1,1))
    
    dataset_train = dataset.iloc[:train.shape[0], :]
    dataset_test = dataset.iloc[train.shape[0]:, :]

    return dataset_train, dataset_test

dataset_train = pd.read_csv('data/train.csv', sep=',')
dataset_test = pd.read_csv('data/test.csv', sep=',')
dataset_train, dataset_test = data_preprocessing(dataset_train, dataset_test)

X_train = dataset_train[features].values
y_train = dataset_train['Survived'].values

Id = dataset_test['PassengerId']
X_submit = dataset_test[features].values

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2)

In [117]:
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone

class MajorityVote(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self, models):
        self.models = models
    
    def fit(self, X, y):
        self.models_ = [clone(x) for x in self.models]

        for model in self.models_:
            model.fit(X,y)
        
        return self
    
    def predict(self, X):
        predict = pd.DataFrame()
        label = 0
        for model in self.models_:
            prediction = model.predict(X)
            predict[str(label)] = prediction
            label += 1
        return predict.mode(axis=1)[0]
        


from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
logi_classifier = LogisticRegression()
k_classifier = KNeighborsClassifier(n_neighbors=2, metric='minkowski', p=1)
svm_classifier = SVC(kernel='linear', random_state=0)
naive_classifier = GaussianNB()
rndm_classifier = RandomForestClassifier(n_estimators=10, criterion='gini', bootstrap=False, max_features=2, min_samples_leaf=1, min_samples_split=20, random_state=0)

majority = MajorityVote(models=(logi_classifier, svm_classifier, naive_classifier, rndm_classifier))
majority.fit(X_train, y_train)
y_pred = majority.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
print(cm)
print(f'accuracy_score = {accuracy_score(y_test,y_pred):.5f}')

[[111   5]
 [ 29  34]]
accuracy_score = 0.81006


In [121]:
# Logistic Regression
from sklearn.linear_model import LogisticRegression
logi_classifier = LogisticRegression()
logi_classifier.fit(X_train, y_train)
y_pred = logi_classifier.predict(X_submit)

from sklearn.metrics import confusion_matrix, accuracy_score
def create_confusion_matrix(y_test, y_pred):
    cm = confusion_matrix(y_test, y_pred)
    print(cm)
    print(f'accuracy_score = {accuracy_score(y_test,y_pred):.5f}')

#create_confusion_matrix(y_test, y_pred)


output = pd.DataFrame({'PassengerId': Id, 'Survived': y_pred})
output.to_csv('my_submission.csv', index=False)
print("Your submission was successfully saved!")

Your submission was successfully saved!


In [109]:
# K-Nearest Neighbors
from sklearn.neighbors import KNeighborsClassifier
k_classifier = KNeighborsClassifier(n_neighbors=2, metric='minkowski', p=1)
k_classifier.fit(X_train, y_train)
y_pred = k_classifier.predict(X_test)

create_confusion_matrix(y_test, y_pred)

[[110   3]
 [ 39  27]]
accuracy_score = 0.76536


In [132]:
# 過学習?のため結果はいまいち。EDAを見直す必要あり?

cat_classifier = CatBoostClassifier(iterations=1000, use_best_model=True, eval_metric='Accuracy', od_pval=0, learning_rate=0.0001, depth=10)
cat_classifier.fit(X_train, y_train, eval_set=(X_test, y_test))
prediction = cat_classifier.predict(X_submit)

#y_pred = cat_classifier.predict(X_test)
output = pd.DataFrame({'PassengerId': Id, 'Survived': prediction})
output.to_csv('my_submission.csv', index=False)
#print("Your submission was successfully saved!")

539326	test: 0.8044693	best: 0.8156425 (1)	total: 6.49s	remaining: 1.89s
774:	learn: 0.8539326	test: 0.8044693	best: 0.8156425 (1)	total: 6.5s	remaining: 1.89s
775:	learn: 0.8539326	test: 0.8044693	best: 0.8156425 (1)	total: 6.51s	remaining: 1.88s
776:	learn: 0.8539326	test: 0.8044693	best: 0.8156425 (1)	total: 6.51s	remaining: 1.87s
777:	learn: 0.8539326	test: 0.8044693	best: 0.8156425 (1)	total: 6.53s	remaining: 1.86s
778:	learn: 0.8539326	test: 0.8044693	best: 0.8156425 (1)	total: 6.54s	remaining: 1.85s
779:	learn: 0.8539326	test: 0.8044693	best: 0.8156425 (1)	total: 6.57s	remaining: 1.85s
780:	learn: 0.8539326	test: 0.8044693	best: 0.8156425 (1)	total: 6.57s	remaining: 1.84s
781:	learn: 0.8539326	test: 0.8044693	best: 0.8156425 (1)	total: 6.59s	remaining: 1.84s
782:	learn: 0.8539326	test: 0.8044693	best: 0.8156425 (1)	total: 6.6s	remaining: 1.83s
783:	learn: 0.8539326	test: 0.8044693	best: 0.8156425 (1)	total: 6.61s	remaining: 1.82s
784:	learn: 0.8539326	test: 0.8044693	best: 0.815

In [110]:
# Support Vector Machine
from sklearn.svm import SVC
svm_classifier = SVC(kernel='linear', random_state=0)
svm_classifier.fit(X_train, y_train)
y_pred = svm_classifier.predict(X_test)

create_confusion_matrix(y_test, y_pred)

[[101  12]
 [ 18  48]]
accuracy_score = 0.83240


In [111]:
# Naive Bayes
from sklearn.naive_bayes import GaussianNB
naive_classifier = GaussianNB()
naive_classifier.fit(X_train, y_train)
y_pred = naive_classifier.predict(X_test)

create_confusion_matrix(y_test, y_pred)

[[98 15]
 [14 52]]
accuracy_score = 0.83799


In [112]:
# Random Forest Classification
from sklearn.ensemble import RandomForestClassifier
rndm_classifier = RandomForestClassifier(n_estimators=10, criterion='gini', bootstrap=False, max_features=2, min_samples_leaf=1, min_samples_split=20, random_state=0)
rndm_classifier.fit(X_train, y_train)
y_pred = rndm_classifier.predict(X_test)

create_confusion_matrix(y_test, y_pred)

[[105   8]
 [ 19  47]]
accuracy_score = 0.84916


In [221]:
# KFold
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
kfold = KFold(n_splits=10, random_state=42)
result = cross_val_score = cross_val_score(RandomForestClassifier(), X_train, y_train, cv = kfold, scoring = 'accuracy')
print(result.mean())

0.8070104025327905


In [222]:
# grid search
'''
from sklearn.model_selection import GridSearchCV
grid_parameters = [
    {'n_estimators': [1, 2, 5, 10, 100, 1000], 
    'criterion': ['gini', 'entropy'],
    'max_features': [1, 2, 5, 10, 20],
    'min_samples_split': [1, 2, 5, 10, 20],
    'min_samples_leaf': [1, 2, 5, 10, 20],
    'bootstrap': [True, False],
    }
]

grid_search = GridSearchCV(RandomForestClassifier(), grid_parameters, cv=5, scoring='accuracy', n_jobs = -1)
grid_search.fit(X_train, y_train)
grid_search.best_params_

SyntaxError: EOF while scanning triple-quoted string literal (<ipython-input-222-da4f042cd10a>, line 16)

In [18]:
# LightGBM
lgb_train = lgb.Dataset(X_train, y_train)
lgb_test = lgb.Dataset(X_test, y_test)
parameter = {
    'objective': 'binary',
    'random_seed': 1234,
    'num_iterations': 100,
    'max_depth': -1,
    'num_leaves':20,
    'max_bin': 500,
    'min_data_in_leaf': 57
}
model = lgb.train(parameter, train_set=lgb_train, valid_sets=lgb_test, num_boost_round=200,
                    early_stopping_rounds=20, verbose_eval=10)


[LightGBM] [Info] Number of positive: 267, number of negative: 401
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003365 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 193
[LightGBM] [Info] Number of data points in the train set: 668, number of used features: 8
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.399701 -> initscore=-0.406713
[LightGBM] [Info] Start training from score -0.406713
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Training until validation scores don't improve for 20 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] 

In [21]:
y_pred = model.predict(X_test)
metric = []
range_index = 10000
for x in range(0, range_index, 1):
    y_adjust = np.where(y_pred > x/range_index, 1, 0)
    metric.append(accuracy_score(y_test, y_adjust))
#create_confusion_matrix(y_test, y_pred)
print(f'\n\nmax accuracy is {max(metric):.4f} at {metric.index(max(metric))/range_index}')

y_pred = np.where(y_pred > 0.5429, 1, 0)
create_confusion_matrix(y_test, y_pred)



max accuracy is 0.8520 at 0.5429
[[142   6]
 [ 27  48]]
accuracy_score = 0.85202


In [223]:
# Predict
test_dataset = pd.read_csv('data/test.csv')
X, y, test_dataset = data_preprocessing(test_dataset)
y_pred = model.predict(X)
y_pred = np.where(y_pred > 0.4128, 1, 0)

output = pd.DataFrame({'PassengerId': test_dataset['PassengerId'], 'Survived': y_pred})
output.to_csv('my_submission.csv', index=False)
print("Your submission was successfully saved!")

Your submission was successfully saved!
